### Import library

In [1]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    CV branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

' Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP\n    Goal is to predict if mutation is SNP or PD\n    CV branch\n    \n    Total samples: 3368\n    2254 PD samples\n    1111 SNP samples\n'

In [2]:
""" Imports the required libraries and packages """

import pandas as pd  #Import for data manipulation in dataframes
import numpy as np  #Array manipulation and calculates mean

import random as rd
import time
import sys
import re

import matplotlib.pyplot as plt
from matplotlib.patches import Patch #CV visualise

import PDB2AC

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )
from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Cross validation to improve hyperparameters
    RandomizedSearchCV,
    KFold,
    StratifiedKFold, # K-fold CV
    GroupKFold,
    StratifiedGroupKFold
        )

from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each

np.set_printoptions(suppress=True, threshold=np.inf, precision=3) #full array printing

KeyboardInterrupt: 

### Split dataset into training and validation sets

In [ ]:
def Train_Test_Split(file):
    """      
    Input:      file             Pre-processed dataset done by PDB2AC script

    Returns:    Training_Set     80% training set split
                Testing_Set      20% testing set split
                labels           Class labels for training set

    80% training and 20% testing split. Writes the data to txt files. Splits are shuffled randomly
    """
    Training_Set, Testing_Set = train_test_split(Cleaned,train_size = 0.8)
    labels = Training_Set['dataset_pd'].astype('int32')
    
    
    Training_file = Training_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)
    Testing_file = Testing_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)

    with open('Training set.txt', 'w') as file: #Writes training data to files
        file.write(Training_file.to_string())
    with open('Testing set.txt', 'w') as file: #Writes testing data to files
        file.write(Testing_file.to_string())


    return Training_Set, Testing_Set, labels

### Initial evaluation

In [ ]:
# def test(Initial_RFC, Input_test, Classes_test):
#     """ Input:  Input_test      Features test data
#                 Classes_test    Class label test data

#         Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
#         are the class labels testing data
#     """

#     Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
#     print(f"              **Initial Evaluation**\n")
#     print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
#     print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


## Group K-fold CV (outer loop)

In [ ]:
def Group_data(Training_Set):
    """      
    Input:      Training_Set     80% training set split

    Returns:    Input_CV         Datastet of input features for training
                Output_CV        Datastet of class labels for training
                Protein_Groups   List of proteins for grouping

    Alphabetically order the identifier column, extract only the 'pdbcode' and make that the identifer column.
    Use this formatted dataset to create the training features, class labels, and group identifiers.
    """
    Input_CV = Training_Set.drop(['dataset_pd'], axis =1)
    Output_CV = Training_Set['dataset_pd'].copy().astype('int32') 
    Protein_Groups = Training_Set['AC_codes'].to_list()

    return Input_CV, Output_CV, Protein_Groups

In [ ]:
def CV(Input_CV, Output_CV, Protein_Groups):
    """      
    Input:      Input_CV             Datastet with input features for training
                Output_CV            Datastet with class labels for training
                Protein_Groups       List of proteins for grouping
            
    Returns:    Input_train     Training features, for each fold
                Classes_train   Ttraining classes, for each fold
                Input_val       Validating features, for each fold
                Classes_val     Validating classes, for each fold

    Group K-fold CV that maintains protein groups, attempts to preserve number of samples of each class 
    for each fold, and ensures protein groups are separated. Creates 5 folds.
    """
        
    CV = GroupKFold(n_splits = 5) #Only shuffles proteins in each group, not groups in fold
    
    Input_train_list = []
    Classes_train_list = []
    Input_val_list = []
    Classes_val_list = []
    
    for train_idx, val_idx in CV.split(Input_CV, Output_CV, Protein_Groups):
        Rd = np.random.randint(time.time()) #Random number from 1 to time since epoch

        Input_train = Input_CV.loc[train_idx]
        Classes_train = Output_CV.loc[train_idx]
        Input_train.drop(['PDB code'], axis = 1, inplace = True) 

        Input_val = Input_CV.loc[val_idx]
        Classes_val = Output_CV.loc[val_idx]
        Input_val.drop(['PDB code'], axis = 1, inplace = True)
        
        Input_train_list.append(Input_train.sample(frac=1, random_state=Rd))
        Classes_train_list.append(Classes_train.sample(frac=1, random_state=Rd))
        Input_val_list.append(Input_val.sample(frac=1, random_state=Rd))
        Classes_val_list.append(Classes_val.sample(frac=1, random_state=Rd))
        
    with open('CV validation data.txt', 'w') as f:
        for number, fold in zip(range(len(Classes_val_list)), Classes_val_list):
            f.write(f"Fold: {number}\n\n{fold.to_string()}\n\n\n")

    return(Input_train_list, Classes_train_list, Input_val_list, Classes_val_list)


### Visualise CV

## Balancing (inner loop)

In [ ]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
    
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [ ]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing for a fold between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [ ]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   input_balance  Array of balanced input training data
                       label_balance
                       
        Create array of the input training data and classes used. Converts from array to dataframe for CV code compatibility.
        The index [i] is the identifier between the two arrays.
    """
    input_balance = []
    label_balance = []
    
    for i in range(len(inData)):
        if usedLines[i]:
            input_balance.append(inData[i])
            label_balance.append(classData[i])

    input_balance = np.stack(input_balance, axis =0)
    label_balance = np.stack(label_balance, axis =0)     
    
#     Dataframe format
#     input_balance = pd.DataFrame(input_balance, columns = ['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'])
#     label_balance = pd.DataFrame(label_balance, columns = ['Class'])
    
    return input_balance, label_balance

### Balance for n folds

In [ ]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [ ]:
def Balance_Folds(BF, inData, classData, minClass, minSize):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    input_balance     Dataframe of balanced training data
                    label_balance
                    
        Returns:    Input_folds       List of balanced arrays of training data
                    Output_folds

        Perform the balance_data() function n number of balancing fold times. Return lists for feature data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    
    for i in range(BF):
        usedLines = balance(inData, classData, minClass, minSize)
        input_balance, label_balance = balance_data(inData, classData, usedLines)
        
        Input_folds.append(input_balance)
        Output_folds.append(label_balance)
        
#     with open('Balanced training data.txt', 'w') as f:
#         for number, fold in zip(range(BF), train_folds):
#             f.write(f"Fold: {number}\n\n{fold}\n\n\n")
            
    return Input_folds, Output_folds

#### RFC hyperparameter tuning

In [ ]:
#  def Hyperparameter(BF, Input_folds, Output_folds):
#     """ Input:      BF                Number of balancing folds needed
#                     Input_folds       List of 5 balanced arrays of training data
#                     Output_folds      List of 5 balanced arrays of training data's labels

#         Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

#         Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
#     """  
#     estimator = RandomForestClassifier()
#     param_grid = {
#                 'n_estimators':np.arange(50,500,50),
#                 'max_depth': np.arange(2, 10, 2),
#                 'max_samples': np.arange(0.2, 1.2, 0.2)
#                   }
#     BF_RFC_HP = []

#     for i in range(BF):
#         HPtuning = RandomizedSearchCV(
#             estimator,
#             param_grid, 
#             scoring = 'balanced_accuracy',
#             cv = 10,
#             n_jobs = 6, #how many cores to run in parallel
#             verbose = 2
#             ).fit(Input_folds[i], Output_folds[i].ravel())
#         BF_RFC_HP.append(HPtuning.best_params_)
    
#     return(BF_RFC_HP)

### Train RFC on the trainings folds

In [ ]:
def BF_fitting(Input_folds, Output_folds): 
    """ Input:
                    Input_train_list      List of 5 balanced arrays for training data
                    Classes_train_list    List of 5 balanced arrays of training data labels
        Returns:    BF_RFC                List of RFC's trained on data in each balancing fold

        Create RFC model that returns probability predictions for each fold, using output of Balance_Folds() as training data
    """    
    BF_RFC = []
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(verbose = 1, n_estimators = 1000)) #Generates a RFC for each fold's training data
        BF_RFC[i].fit(Input_folds[i], Output_folds[i]) #Fits the RFC to each folds' training data
        
    return BF_RFC

#### Validate each RFC on validation set, for each fold

In [ ]:
def BF_validate(BF_RFC, ValData):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_val_list  Unseen validation data fold from CV fold
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    
    Prob_matrix = [] #Empty list
    for i in range(len(BF_RFC)):
        Prob = BF_RFC[i].predict_proba(ValData.values)
        Prob_matrix.append(Prob)   
            
    with open('Validation fold probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

### Weighted voting

In [ ]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = [] #Empty list
    Sc_SNP = [] #Empty list
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     
    
    Vote_arr = [] #Empty list

    for i in range(len(Sum_PD)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list

        Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array
        Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array

    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


### Final confidence

In [ ]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out       Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [ ]:
def evalutation(Vallabel, Final_vote, S_Out):
    """ Input:      Classes_val_list   Validation label data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on validation data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
    print(f"{classification_report(Vallabel, Output_pred)}\nMCC                {matthews_corrcoef(Vallabel, Output_pred)}")

    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )


### Main Program

In [3]:
file = "AC_dataset.csv"
Training_Set, Testing_Set, labels = Train_Test_Split(file)

Input_CV, Output_CV, Protein_Groups = Group_data(Training_Set)
Input_train_list, Classes_train_list, Input_val_list, Classes_val_list = CV(Input_CV, Output_CV, Protein_Groups)

for i in range(len(Input_train_list)): #For every CV fold
    classData = Classes_train_list[i].to_numpy()
    inData = Input_train_list[i].to_numpy()
    ValData = Input_val_list[i]
    Vallabel = Classes_val_list[i]

    minClass, minSize, maxSize = find_minority_class(classData)
    BF = Balance_ratio(maxSize, minSize)
    Input_folds, Output_folds = Balance_Folds(BF, inData, classData, minClass, minSize) #balance() and balance_data() functions are called under this

    BF_RFC = BF_fitting(Input_folds, Output_folds)
    Prob_matrix = BF_validate(BF_RFC, ValData)

    Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix, BF)
    S_Out = Final_score(Sum_PD, Sum_SNP, BF)

    evalutation(Vallabel, Final_vote, S_Out)

NameError: name 'Clean_data' is not defined